In [1]:
import matplotlib.pyplot as plt
from hmf import MassFunction
import numpy as np
import pandas as pd
from astropy.cosmology import WMAP7, FlatLambdaCDM
from scipy.constants import G
import pickle
import fitsio as fio
import images
from scipy.stats import lognorm, gennorm
import tutorial.synthetic.tools as tools
import tutorial.synthetic.render.frame as frame
import tutorial.synthetic.render.render as render
# File directory: inside galaxy clustering. Within same folder as gal_gg_nmgy.pkl/
# Has a tutorial folder with synthetic in. If following my tutorial you only need to paste gal_gmm_nmgy.pkl or modify the path to there.
size = 1000
width = 5000
height = 5000
mean_sources = 0.005
flux_min = 500
flux_max = 350000
galaxy_flux_alpha = 0.48
t_min = 0.15
t_max = 5
t_alpha = 3
shape_min = 0.01
shape_max = 0.7
shape_alpha = 0.2
bands = ["G", "R", "I", "Z"]
n_bands = 4
reference_band = 1
ra_cen = 50.64516228577292
dec_cen = -40.228830895890404

In [2]:
# Sample mass
mass_min = 10**14 * 1.989*10**33 # Minimum value of the range solar mass
mass_max = 10**15.5 * 1.989*10**33 # Maximum value of the range
hmf = MassFunction()
hmf.update(Mmin=14, Mmax = 15.5) 
mass_func = hmf.dndlnm
mass_sample = []
while(len(mass_sample) < size):
    index = np.random.randint(0, len(mass_func))
    prob = (mass_func/sum(mass_func))[index]
    if np.random.random() < prob:
        mass_sample.append((mass_max-mass_min)/len(mass_func)*(index+np.random.random()) + mass_min)

In [3]:
# Sample redshift
redshift_samples = np.random.uniform(0.2, 0.5, size)

In [4]:
# Calculate individual zis
def zi_based_Z(z, mass, size):
    delta = (WMAP7.H(z).value/100*mass/(10**15*1.989*10**33))**(1/3)*1082.9
    # light speed 
    c = 899377.37
    speed_mean = (c*(1+z)**2 - c)/((1+z)**2 + 1)
    temp = np.random.normal(speed_mean, delta, size)
    temp = [max(0, np.sqrt((c+x)/(c-x)) - 1) for x in temp]
    return temp

# Calculate radius based on mass and redshift
def R_based_Z_M(mass, z):
    pho_z = WMAP7.critical_density(z).value
    return (mass/(4/3*np.pi*pho_z))**(1/3)

In [5]:
# Get radius based on mass and redshift in cm.
radius_samples = []
for i in range(size):
    radius_samples.append(R_based_Z_M(mass_sample[i], redshift_samples[i])/(3.086*10**24))

In [6]:
# Determine # of galaxies
n_galaxy_cluster = []
for i in range(size):
    n_galaxy_cluster.append(int(((mass_sample[i]/(1.989*10**33))/(2*10**13))**(1/1.2)*20))

In [7]:
# Sample center
center_sample = []
x_coords = np.random.uniform(width * 0.3, width * 0.7, size)
y_coords = np.random.uniform(height * 0.3, height * 0.7, size)
center_sample = np.vstack((x_coords, y_coords)).T

In [8]:
scale_pixels_per_au = 100  
radius_samples = [x*scale_pixels_per_au for x in radius_samples]  

galaxy_locs_cluster = []
for i in range(size):
    center_x, center_y = center_sample[i]
    samples = []
    while(len(samples) < int(n_galaxy_cluster[i])):
        angles = np.random.uniform(0, 2 * np.pi, 1)
        radii = np.random.uniform(0, radius_samples[i], 1)  
        sampled_x = float(center_x + radii * np.cos(angles))  
        sampled_y = float(center_y + radii * np.sin(angles))
        if sampled_x >= 0 and sampled_x < width and sampled_y >= 0 and sampled_y < height:
            samples.append([sampled_x, sampled_y])
    while(len(samples) < 1.5*int(n_galaxy_cluster[i])):
        angles = np.random.uniform(0, 2 * np.pi, 1)
        radii = np.random.uniform(radius_samples[i] + 0.001, 2*radius_samples[i], 1)  
        sampled_x = float(center_x + radii * np.cos(angles))  
        sampled_y = float(center_y + radii * np.sin(angles))
        if sampled_x >= 0 and sampled_x < width and sampled_y >= 0 and sampled_y < height and np.random.random() > (((sampled_x - center_x)**2 + (sampled_y - center_y)**2)**0.5 - radius_samples[i])/radius_samples[i]:
            samples.append([sampled_x, sampled_y])
    galaxy_locs_cluster.append(samples)

In [9]:
n_galaxy = np.random.poisson(mean_sources*width*height/36, size)

In [10]:
for i in range(size):
    n_galaxy[i] -= n_galaxy_cluster[i]

In [11]:
galaxy_locs = []
for i in range(size):
    x = np.random.uniform(0, width, n_galaxy[i])
    y = np.random.uniform(0, height, n_galaxy[i])
    galaxy_locs.append(np.column_stack((x, y)))

In [12]:
def cartesian2geo(coordinates, sky_center=(ra_cen, dec_cen), pixel_scale=0.2, image_offset=(2499.5, 2499.5)):
    geo_coordinates = []
    for i in range(len(coordinates)):
        temp = []
        for j in range(len(coordinates[i])):
            ra = (coordinates[i][j][0] - image_offset[0])*pixel_scale / (60*60) + sky_center[0]
            desc = (coordinates[i][j][1] - image_offset[1])*pixel_scale / (60*60) + sky_center[1]
            temp.append((ra, desc))
        geo_coordinates.append(temp)
    return geo_coordinates

In [13]:
geo_galaxy = cartesian2geo(galaxy_locs)
geo_galaxy_cluster = cartesian2geo(galaxy_locs_cluster)

In [14]:
def draw_truncated_pareto(alpha, min_x, max_x, n_samples):
    # draw pareto conditioned on being less than f_max
    u_max = 1 - (min_x / max_x) ** alpha
    uniform_samples = np.random.random(int(n_samples)) * u_max
    return min_x / (1.0 - uniform_samples) ** (1 / alpha)

In [15]:
t_size_samples = []
flux_samples = []
for i in range(size):
    total_element = len(galaxy_locs[i]) + len(galaxy_locs_cluster[i])
    t_size_samples.append(lognorm.rvs(s=0.64, loc=0.017, scale=0.23, size=total_element))
    mag_samples = 25 - np.random.exponential(1.3, total_element)
    for j in range(len(mag_samples)):
        while(mag_samples[j] < 15.75):
            mag_samples[j] = (25 - np.random.exponential(1.3, 1))[0]
        mag_samples[j] = tools.toflux(mag_samples[j])
        if j <= len(galaxy_locs_cluster[i]):
            mag_samples[j] *= (1+zi_based_Z(redshift_samples[i], mass_sample[i], 1)[0])
        else:
            mag_samples[j] *= (1+np.random.uniform(0, 0.3))
    flux_samples.append(mag_samples)

In [16]:
G1_size_samples = []
G2_size_samples = []
for i in range(size):
    total_element = len(galaxy_locs[i]) + len(galaxy_locs_cluster[i])
    G1_size_samples.append(gennorm.rvs(0.6, 0, 0.035, total_element))
    G2_size_samples.append(gennorm.rvs(0.6, 0, 0.032, total_element))
    for j in range(total_element):
        while G1_size_samples[i][j]**2 + G2_size_samples[i][j]**2 >= 1 or G1_size_samples[i][j] >= 0.8 or G2_size_samples[i][j] >= 0.8:
            G1_size_samples[i][j] = gennorm.rvs(0.6, 0, 0.035, 1)[0]
            G2_size_samples[i][j] = gennorm.rvs(0.6, 0, 0.032, 1)[0]

In [17]:
# This gives ["u", "g", "r", "i", "z"] 
with open("gal_gmm_nmgy.pkl", "rb") as f:
    gmm_gal = pickle.load(f)

/home/shihangl/bliss/.venv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GaussianMixture from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
def galaxy_flux_ratio(size, gmm_gal):
    # ["G", "R", "I", "Z"]
    flux_logdiff, _ = gmm_gal.sample(size)
    flux_logdiff = flux_logdiff[:][:, 1:]
    flux_logdiff = np.clip(flux_logdiff, -2.76, 2.76)        
    flux_ratio = np.exp(flux_logdiff)
    flux_prop = np.ones((flux_logdiff.shape[0], n_bands))
    for band in range(reference_band - 1, -1, -1):
        flux_prop[:, band] = flux_prop[:, band + 1] / flux_ratio[:, band]
    for band in range(reference_band + 1, n_bands):
        flux_prop[:, band] = flux_prop[:, band - 1] * flux_ratio[:, band - 1]
    return flux_prop

In [19]:
def flux2mag(flux, constant=29.5):
    return -2.5*np.log10(flux) + constant

In [20]:
def make_catalog(gmm_gal, flux_samples, t_size_samples, G1_size_samples, G2_size_samples,geo_galaxy, geo_galaxy_cluster, galaxy_locs, galaxy_locs_cluster):
    res = []
    for i in range(len(flux_samples)):
        ratios = galaxy_flux_ratio(len(geo_galaxy_cluster[i]) + len(geo_galaxy[i]), gmm_gal)
        fluxes = np.array(flux_samples[i])[:, np.newaxis] * np.array(ratios)
        mock_catalog = pd.DataFrame()
        mock_catalog["RA"] = np.append(np.array(geo_galaxy_cluster[i])[:, 0], np.array(geo_galaxy[i])[:, 0])
        mock_catalog["DEC"] = np.append(np.array(geo_galaxy_cluster[i])[:, 1], np.array(geo_galaxy[i])[:, 1])
        mock_catalog["X"] = np.append(np.array(galaxy_locs[i])[:, 0], np.array(galaxy_locs_cluster[i])[:, 0])
        mock_catalog["Y"] = np.append(np.array(galaxy_locs[i])[:, 1], np.array(galaxy_locs_cluster[i])[:, 1])
        mock_catalog["FLUX_R"] = fluxes[:, 1]
        mock_catalog["MAG_R"] = -2.5*np.log10(mock_catalog["FLUX_R"]) + 30
        mock_catalog["FLUX_G"] = fluxes[:, 0]
        mock_catalog["MAG_G"] = -2.5*np.log10(mock_catalog["FLUX_G"]) + 30
        mock_catalog["FLUX_I"] = fluxes[:, 2]
        mock_catalog["MAG_I"] = -2.5*np.log10(mock_catalog["FLUX_I"]) + 30
        mock_catalog["FLUX_Z"] = fluxes[:, 3]
        mock_catalog["MAG_Z"] = -2.5*np.log10(mock_catalog["FLUX_Z"]) + 30
        mock_catalog["TSIZE"] = t_size_samples[i]
        mock_catalog["FRACDEV"] = 0
        mock_catalog["G1"] = G1_size_samples[i]
        mock_catalog["G2"] = G2_size_samples[i]
        res.append(mock_catalog)
    return res

In [21]:
res = make_catalog(gmm_gal, flux_samples, t_size_samples, G1_size_samples, G2_size_samples,geo_galaxy, geo_galaxy_cluster, galaxy_locs, galaxy_locs_cluster)

In [22]:
# file_name_tag = 'test_canvas_dc2_' #
# # Pick one you want to see
# index = 1
# mock_catalog = res[index]
# stds = np.array([2.509813, 5.192254, 8.36335, 15.220351]) / 1.3
# for i, band in enumerate(("g", "r", "i")):
#     name =  file_name_tag + band
#     print(name)
#     fr = frame.Frame(mock_catalog.to_records(), band=band, name=name,
#                      center=(ra_cen, dec_cen), noise_std=stds[i], canvas_size=5000, )
#     fr.render(nprocess=8) 

In [23]:
# ims_all = []
# for i, band in enumerate(("g", "r", "i")):
#     name = file_name_tag + band + ".fits"
#     tmp = fio.read(name)
#     print(name)
#     ims_all.append(tmp)

In [24]:
# fig = plt.figure(figsize=(12, 12))
# ax = fig.add_subplot(111)
# factor = 0.001
# scales = np.array([1., 1.2, 2.5]) * factor
# nonlinear = 0.12
# clip = 0

# pad = 0
# obs_im = images.get_color_image(ims_all[2],
#                                 ims_all[1],
#                                 ims_all[0],
#                                 nonlinear=nonlinear, clip=clip, scales=scales)  
# print(obs_im.max())
# ax.imshow(obs_im * 2, origin='upper')
# ax.plot(center_sample[index][0], center_sample[index][1], 'rx', markersize=10, markeredgewidth=2)
# # ax.set_title("cluster + field rendered together")

# ax.set_xlabel("X [pix]")
# ax.set_ylabel("Y [pix]")
# fig.savefig("test.png", bbox_inches='tight')

In [25]:
tiles_width = 25
tiles_height = 25
def to_tiles(tiles_height, tiles_width, center_sample, mass_sample, width, height):
     tiles = []
     for i in range(len(mass_sample)):
          tiles_X = width // tiles_width
          tiles_Y = height // tiles_height
          x_index = int(center_sample[i][0] // tiles_width)
          y_index = int(center_sample[i][1] // tiles_height)
          cluster_grid = [[0 for _ in range(tiles_X)] for _ in range(tiles_Y)]
          cluster_grid[x_index][y_index] = 1
          mass_grid = [[0 for _ in range(tiles_X)] for _ in range(tiles_Y)]
          mass_grid[x_index][y_index] = mass_sample[i]
          coordinate_grid = [[(0, 0) for _ in range(tiles_X)] for _ in range(tiles_Y)]
          coordinate_grid[x_index][y_index] = (center_sample[i][0]%tiles_width, center_sample[i][1]%tiles_height)
          temp = {}
          temp["mass"] = mass_grid
          temp["coordinate"] = coordinate_grid
          temp["cluster"] = cluster_grid
          tiles.append(temp)
     return tiles
          

In [26]:
tiles = to_tiles(tiles_height, tiles_width, center_sample, mass_sample, width, height)

In [27]:
import threading
import os
def catalog_render(catalogs, start_index, tiles):
    print(str(start_index) + " has begun.")
    for i in range(len(catalogs)):
        mock_catalog = catalogs[i]
        stds = np.array([2.509813, 5.192254, 8.36335, 15.220351]) / 1.3
        file_name_tag = str(i+start_index)
        for j, band in enumerate(("g", "r", "i")):
            name =  file_name_tag + "_"+ band
            print(name)
            fr = frame.Frame(mock_catalog.to_records(), band=band, name=name,
                            center=(ra_cen, dec_cen), noise_std=stds[j], canvas_size=5000, )
            fr.render(nprocess=8) 
        ims_all = []
        for j, band in enumerate(("g", "r", "i")):
            name = file_name_tag + "_" + band + ".fits"
            tmp = fio.read(name)
            os.remove(name)
            os.remove(file_name_tag + "_" + band + "_epsf.fits")
            ims_all.append(tmp)
        fig = plt.figure(figsize=(12, 12))
        ax = fig.add_subplot(111)
        factor = 0.001
        scales = np.array([1., 1.2, 2.5]) * factor
        nonlinear = 0.12
        clip = 0
        obs_im = images.get_color_image(ims_all[2],
                                        ims_all[1],
                                        ims_all[0],
                                        nonlinear=nonlinear, clip=clip, scales=scales)  
        print(obs_im.max())
        ax.imshow(obs_im * 2, origin='upper')

        ax.set_xlabel("X [pix]")
        ax.set_ylabel("Y [pix]")
        fig.savefig("data/" + file_name_tag + ".png", bbox_inches='tight')
        filehandler = open("data/" + file_name_tag + "_catalog.pkl", 'wb') 
        pickle.dump(tiles[i], filehandler)

In [28]:
for i in range(5):
    x = threading.Thread(target = catalog_render, args = (res[i*size//5:(i+1)*size//5], i*size//5, tiles[i*size//5:(i+1)*size//5], ))
    x.start()

0 has begun.
0_g
200 has begun.
200_g
400 has begun.
400_g
600 has begun.
600_g
800 has begun.
800_g


starting postage stamp calculations in 8 processes
starting postage stamp calculations in 8 processes
starting postage stamp calculations in 8 processes


starting postage stamp calculations in 8 processes
starting postage stamp calculations in 8 processes
starting postage stamp calculations in 8 processes
400_r
starting postage stamp calculations in 8 processes
600_r
starting postage stamp calculations in 8 processes
800_r
starting postage stamp calculations in 8 processes
0_r
starting postage stamp calculations in 8 processes
200_r
starting postage stamp calculations in 8 processes
400_i
starting postage stamp calculations in 8 processes
600_i
starting postage stamp calculations in 8 processes
800_i
starting postage stamp calculations in 8 processes
0_i
starting postage stamp calculations in 8 processes
200_i
starting postage stamp calculations in 8 processes
1.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1.0
1.0


...

1.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1.0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


401_g
starting postage stamp calculations in 8 processes
401_r
starting postage stamp calculations in 8 processes
